In [1]:
from typing import List, Dict, Tuple
from collections import defaultdict as dd
import os
import re

import pandas as pd
import numpy as np
from scipy import stats
from scipy.sparse import lil_matrix, coo_matrix
from sklearn.linear_model import LogisticRegression
from tqdm.notebook import tqdm

In [2]:
FOLDER = 'chgk'
NoReturn = None
EPS = 0.001

In [46]:
players = pd.read_pickle(os.path.join(FOLDER, "players.pkl"))
tournaments = pd.read_pickle(os.path.join(FOLDER, "tournaments.pkl"))

In [45]:
results = pd.read_pickle(os.path.join(FOLDER, "results.pkl"))

### Tournaments dataset

In [47]:
tournaments = pd.DataFrame(tournaments.values())
tournaments['dateStart'] = pd.to_datetime(tournaments['dateStart'], format='%Y-%m-%d', utc=True)
tournaments['dateEnd'] = pd.to_datetime(tournaments['dateEnd'], format='%Y-%m-%d', utc=True)

In [48]:
tournaments = tournaments.set_index('id')
tournaments

,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
id,,,,,,,,
1,Чемпионат Южного Кавказа,2003-07-24 20:00:00+00:00,2003-07-26 20:00:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,Летние зори,2003-08-08 20:00:00+00:00,2003-08-08 20:00:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
3,Турнир в Ижевске,2003-11-21 21:00:00+00:00,2003-11-23 21:00:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,Чемпионат Украины. Переходной этап,2003-10-10 20:00:00+00:00,2003-10-11 20:00:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
5,Бостонское чаепитие,2003-10-09 20:00:00+00:00,2003-10-12 20:00:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
...,...,...,...,...,...,...,...,...
6481,Онлайн: 15:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-05 12:00:00+00:00,2020-05-05 15:00:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 12, '2': 12, '3': 12}"
6482,Онлайн: 19:00 Зелёный шум,2020-05-07 16:00:00+00:00,2020-05-07 18:30:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 13, '2': 13, '3': 13}"
6483,Онлайн: 19:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-08 16:00:00+00:00,2020-05-08 18:30:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 12, '2': 12, '3': 12}"


I break down dataset into 2 parts: <br>
* Train - 2019 year <br>
* Test - 2020 year <br>

In [49]:
train_tournaments = tournaments[(tournaments['dateStart'] >= '2019-01-01') & (tournaments['dateStart'] < '2020-01-01')]
test_tournaments = tournaments[(tournaments['dateStart'] >= '2020-01-01') & (tournaments['dateStart'] < '2021-01-01')]

In [50]:
train_tournaments = train_tournaments[train_tournaments.index.isin(results.keys())]
test_tournaments = test_tournaments[test_tournaments.index.isin(results.keys())]

In [51]:
test_tournaments

,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
id,,,,,,,,
4628,Семь сорок,2020-12-30 13:00:00+00:00,2020-12-30 13:00:00+00:00,"{'id': 3, 'name': 'Синхрон'}",None,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",{'dateRequestsAllowedTo': '2020-12-30T23:55:00...,"{'1': 12, '2': 12, '3': 12}"
4957,Синхрон Биркиркары,2020-02-20 21:00:00+00:00,2020-02-27 20:00:00+00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 2421, 'name': 'Ася', 'patronymic': 'Се...",{'dateRequestsAllowedTo': '2020-02-27T18:00:00...,"{'1': 13, '2': 13, '3': 13}"
5151,Яровой,2020-08-01 11:00:00+00:00,2020-08-05 11:00:00+00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 22325, 'name': 'Михаил', 'patronymic':...",{'dateRequestsAllowedTo': '2020-07-24T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
5414,Синхрон северных стран,2020-01-03 16:00:00+00:00,2020-01-10 16:00:00+00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2020-01-10T23:59:00...,"{'1': 12, '2': 12, '3': 12}"
5477,Онлайн: Синхрон Урюбджирова,2020-04-18 16:00:00+00:00,2020-04-30 16:00:00+00:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 91324, 'name': 'Эрдни', 'patronymic': ...",{'dateRequestsAllowedTo': '2020-04-30T23:55:00...,"{'1': 12, '2': 12, '3': 12}"
...,...,...,...,...,...,...,...,...
6481,Онлайн: 15:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-05 12:00:00+00:00,2020-05-05 15:00:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 12, '2': 12, '3': 12}"
6482,Онлайн: 19:00 Зелёный шум,2020-05-07 16:00:00+00:00,2020-05-07 18:30:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 13, '2': 13, '3': 13}"
6483,Онлайн: 19:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-08 16:00:00+00:00,2020-05-08 18:30:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 12, '2': 12, '3': 12}"


### Players dataset 

In [52]:
players = pd.DataFrame(players.values())

In [53]:
players = players.set_index('id')

In [54]:
players

,name,patronymic,surname
id,,,
1,Алексей,None,Абабилов
10,Игорь,,Абалов
11,Наталья,Юрьевна,Абалымова
12,Артур,Евгеньевич,Абальян
13,Эрик,Евгеньевич,Абальян
...,...,...,...
224700,Артём,Евгеньевич,Садов
224701,Даниил,Олегович,Трефилов
224702,Владимир,Араратович,Басенцян


Отфильтруем турниры, таким образом, чтобы в "рабочие" турниры попали только те, команды которых имеют не побитую маску ответом на вопросы. А именно, если в маске встречается X, то просто пропускаем такой вопрос, а остальные приводим к целому числу. Если же так получилось, что в команде нет игроков вообще, либо маска полностью невалидная, то игнорируем такую команду как будто её исключили из турнира за странное поведение. Может случится так, что на всём турнире не нашлось команд, маски которых удовлетворяют условию выше, в таком случае такой турнир будет исключён из рассмотрения. <br>
Откорректируем рейтинги команд простой последовательной нумерацией(без учёта ничьи) в турнирах, где были отсеяны некоторые команды.

In [55]:
def filter_valid_rating(tour_res, tour_indexes) -> NoReturn:
    invalids_num = 0
    
    for indx in tour_indexes:
        tour_capacity = len(tour_res[indx])
        valid_teams = []
        for team in tour_res[indx]:
            if team.get('mask'):
                updated_mask = list(map(int, [symbol for symbol in team['mask'] if symbol.isdigit()]))
                if updated_mask and team['teamMembers']:
                    team['mask'] = updated_mask 
                    valid_teams.append(team)
        
        if len(valid_teams) == 0:
            print(f'Tournament ({tournaments.loc[indx, "name"]}) consists of invalid teams...')
            invalids_num += 1
            del tour_res[indx]
            continue
        
        if len(valid_teams) != tour_capacity:
            print(f"Tournament ({tournaments.loc[indx, 'name']}) is invalid, but {len(valid_teams)} teams are valid")
            rating = 1
            for team in valid_teams:
                team['rating'] = rating
                rating += 1
            tour_res[indx] = valid_teams
    
    return tour_res

Если в данных есть некоррекные маски, то будем их выбрасывать и корректировать рейтинги команд в соответствии с исправлениями, но, к сожалению, непонятно как при этом учитывать рейтинги, где была ничья

In [56]:
results = filter_valid_rating(results, list(train_tournaments.index) + list(test_tournaments.index))

Tournament (ОВСЧ. 6 этап) is invalid, but 907 teams are valid
Tournament (Синхрон-lite. Выпуск XXIII) is invalid, but 485 teams are valid
Tournament (Поволжская лига) is invalid, but 5 teams are valid
Tournament (Синхронный Кубок Ростова - общий зачёт) is invalid, but 34 teams are valid
Tournament (Школьный чемпионат Украины) consists of invalid teams...
Tournament (Город грехов) is invalid, but 120 teams are valid
Tournament (Славянка без раздаток. Общий зачёт) is invalid, but 582 teams are valid
Tournament (Гран-при Славянки. Общий зачёт) is invalid, but 561 teams are valid
Tournament (Вторая лига чемпионата Харьковской области) consists of invalid teams...
Tournament (Первая лига чемпионата Харьковской области) consists of invalid teams...
Tournament (Высшая лига чемпионата Харьковской области) consists of invalid teams...
Tournament (Молодёжный чемпионат Нижегородской области) is invalid, but 22 teams are valid
Tournament (Октавы: Гала-турнир. Лига Наций: Прибалтика) consists of in

In [67]:
train_tournaments = train_tournaments[train_tournaments.index.isin(results.keys())]
test_tournaments = test_tournaments[test_tournaments.index.isin(results.keys())]

In [68]:
def get_tour_statistics(tour_res, tour_indexes):
    player_ans_correct = dd(int)
    tour_quest_quantity = dict()
    
    for tour_indx in tour_indexes:
        max_size = 0
        for team in tour_res[tour_indx]:
            quest_size = len(team['mask'])
            if max_size < quest_size:
                max_size = quest_size
                
            for member in team['teamMembers']:
                player_id = member['player']['id']
                player_correct_answers = sum(team['mask'])
                player_ans_correct[player_id] += player_correct_answers
                
        tour_quest_quantity[tour_indx] = max_size
            
    return player_ans_correct, tour_quest_quantity

In [69]:
train_player_ans_correct, train_tour_quest_quantity = get_tour_statistics(results, list(train_tournaments.index))
test_player_ans_correct, test_tour_quest_quantity = get_tour_statistics(results, list(test_tournaments.index))

Заметим, что в данных есть игроки, которые играли в 2019-ом, но не играли в 2020-ом. Подумаем, что делать с такими "кадрами" дальше 

In [70]:
set(train_player_ans_correct.keys()) == set(test_player_ans_correct.keys())

False

In [71]:
print(f'Sum of questions in 2019 is {sum(train_tour_quest_quantity.values())}')
print(f'Sum of questions in 2020 is {sum(test_tour_quest_quantity.values())}')
print(f'Num of players in 2019 is {len(train_player_ans_correct)}')
print(f'Num of players in 2020 is {len(test_player_ans_correct)}')

Sum of questions in 2019 is 33375
Sum of questions in 2020 is 7753
Num of players in 2019 is 59271
Num of players in 2020 is 28301


In [72]:
player_idx = {player:idx for idx, player in enumerate(train_player_ans_correct.keys())}

Обучим логистическую регрессию, где её первыми весами будут выступать силы каждого игрока из тренировочной выборка + обучим сложность каждого вопроса из тренировочной выборки. <br>
Каждое наблюдение будет состоять из игрока и вопроса, на который его команда (не)ответила. Будет считать, что игрок ответил на вопрос, если на него ответила команда в целом, так как проверить кто именно ответил на вопрос не представляется возможным. <br>
Метками будут выступать булевы ответы на вопрос: 1 или 0, где 1- команда ответила на данный вопрос и 0 в противном случае <br>
Заполняем разрежженную матрицу Х и y

In [73]:
def get_player_answers(results, tour_quest_sizes):
    col_step = 0
    player_answer = dd(list)
    num_answers = 0
    the_biggest = -1
    for tour_idx, quest_size in tour_quest_sizes.items():
        for team in results[tour_idx]:
            for quest_idx, answer in enumerate(team['mask']):
                for member in team['teamMembers']:
                    player_id = member['player']['id']
                    player_answer[player_id].append((len(player_idx) + col_step + quest_idx, answer))
                    num_answers += 1
                    if len(player_idx) + col_step + quest_idx > the_biggest:
                        the_biggest = len(player_idx) + col_step + quest_idx
        col_step += quest_size
    return num_answers, player_answer


def fill_X_y(player_answers):
    row_id = 0
    rows = []
    cols = []
    data = []
    y = []
    for idx, answers in player_answers.items():
        for question_id, label in answers:
            rows.extend([row_id, row_id])
            cols.extend([question_id, player_idx[idx]])
            data.extend([1, 1])
            y.append(label)
            row_id += 1
    
    y = np.array(y)
    rows = np.array(rows)
    cols = np.array(cols)
    data = np.array(data)
    X = coo_matrix((data, (rows, cols)),
                   shape=(num_answers, len(player_idx) + sum(train_tour_quest_quantity.values())),
                   dtype=np.int8)
    return X, y

In [74]:
%%time

num_answers, player_answers = get_player_answers(results, train_tour_quest_quantity)
X, y = fill_X_y(player_answers)
print(f'X dim = {X.shape}', f'Y dim = {y.shape}')
del player_answers

X dim = (21008480, 92646) Y dim = (21008480,)
Wall time: 36 s


In [75]:
class CustomLogReg(LogisticRegression):
    
    def fit(self, X, y):
        super().fit(X, y)
        self.weights = self.coef_[0]
    
    def fine_tune_params(self, X, y, lr, max_iter=50):
        X_T = X.T
        ds_size = X.shape[0]
        losses = []
        prev_loss = 10
        for iteration in tqdm(range(max_iter)):
            preds = self.predict_probas(X)
            loss = self.log_loss(y, preds)
            grad = X_T @ (preds - y) / ds_size
            self.weights -= lr * grad
            losses.append(loss)
            
            if iteration % 10 == 0:
                if np.mean(losses) < prev_loss:    
                    prev_loss = np.mean(losses)
                    lr /= 10
                    print(f'LR={lr}')
                    print(f'AVGLogLoss={np.mean(losses)}')
                    losses = []
                else:
                    print("EarlyStopping")
                    break
    
    def predict_probas(self, X):
        logits = X @ self.weights
        return 1 / (1 + np.exp(-logits))
        
    @staticmethod
    def log_loss(y, p):
        return -np.mean(y * np.log(p) + (1 - y) * np.log(1 - p))

In [76]:
%%time

clf = CustomLogReg(random_state=41, solver='saga', fit_intercept=True, n_jobs=-1, multi_class='ovr', tol=0.1)
clf.fit(X, y)

Wall time: 2min 17s


In [77]:
player_ratings = clf.coef_[0, :len(player_idx)]

In [78]:
%%time

data = []

for player_id, num_corr_answers in train_player_ans_correct.items():
    player_info = players.loc[player_id]
    meta_info = {
        'rating': player_ratings[player_idx[player_id]],
        'true_answers': num_corr_answers,
        'name': player_info['name'],
        'surname': player_info['surname'],
        'patronymic': player_info['patronymic'],
    }
    data.append(meta_info)

pred_rating = pd.DataFrame(data)

Wall time: 3.76 s


Рейтинги топ игроков по мнению логистической регрессии. Для оценки силы каждого игрока брались соответствующие веса модели.

In [79]:
pred_rating.sort_values(by=['rating'], ascending=False).head(30)

,rating,true_answers,name,surname,patronymic
8222,4.100847,1980,Максим,Руссо,Михайлович
5957,4.015964,2409,Александра,Брутер,Владимировна
1209,3.984316,3242,Иван,Семушин,Николаевич
155,3.794658,4000,Артём,Сорожкин,Сергеевич
8244,3.777393,3202,Сергей,Спешков,Леонидович
1208,3.773779,2843,Михаил,Савченков,Владимирович
1210,3.630585,1279,Станислав,Мереминский,Григорьевич
3,3.625982,1548,Сергей,Николенко,Игоревич
20010,3.555684,1172,Илья,Новиков,Сергеевич
5,3.551417,849,Ирина,Прокофьева,Сергеевна


In [84]:
def predict_team_score(members, player_ratings):
    member_scores = []
    for member in members:
        member_id = member['player']['id']
        
        if player_idx.get(member_id):
            score = player_ratings[player_idx[member_id]]
        else:
            score = 0
        member_scores.append(score)
    return sum(member_scores) / len(members)

In [139]:
def calculate_local_corr(pred: List[Tuple[str, int]], act: Dict[str, int]):
    act_arr = []
    pred_arr = []
    
    for name, pred_rating in pred:
        act_arr.append(act[name])
        pred_arr.append(pred_rating)
    
    spearman_corr = stats.spearmanr(act_arr, pred_arr).correlation
    kendalltau, _ = stats.kendalltau(act_arr, pred_arr)
    
    return spearman_corr, kendalltau

In [143]:
def calculate_average_correlations(results, player_ratings, indexes, print_axillary_info=True):
    spearman_corrs = []
    kendalltaus = []
    
    for tour_idx in indexes:
        true_team_ratings = {}
        pred_team_scores = []
        tour_name = tournaments.loc[tour_idx, 'name']
        for team in results[tour_idx]:
            true_rating = team['position']
            pred_score = predict_team_score(team['teamMembers'], player_ratings)
            true_team_ratings[team['team']['name']] = true_rating
            pred_team_scores.append((team['team']['name'], pred_score))
        
        sorted_scores = sorted(pred_team_scores, key=lambda x: x[1], reverse=True)
        pred_team_rating = [(name, idx + 1) for idx, (name, score) in enumerate(sorted_scores)]
        
        spearman_corr, kendalltau = calculate_local_corr(pred_team_rating, true_team_ratings)
        
        if pd.isnull(spearman_corr) or pd.isnull(kendalltau):
            continue
            
        spearman_corrs.append(spearman_corr)
        kendalltaus.append(kendalltau)
            
        if print_axillary_info:
            print(f'Tournament-{tour_name}',
                  f'Spearman corr={spearman_corr}',
                  f'Kendall corr={kendalltau}\n', sep='\n')
    
    avg_spearman_corr = sum(spearman_corrs) / len(spearman_corrs)
    avg_kendalltau = sum(kendalltaus) / len(kendalltaus)
    print(f"Avg Spearman correlation={avg_spearman_corr}",
          f'Avg Kendall correlation={avg_kendalltau}\n', sep='\n')

Чтобы оценить результаты турниров из тестовой выборки будем делать следующее: <br>
Возьмём веса каждого игрока в команде и сложим их, после чего усредним по кол-ву человек. Это будет "сила" команды, после чего отранжируем команды по силе и посчитаем корреляции итоговых результатов с предсказанными.

In [145]:
calculate_average_correlations(results, player_ratings, list(test_tour_quest_quantity.keys()), True)

Tournament-Синхрон Биркиркары
Spearman corr=0.7458270508091838
Kendall corr=0.5801838669498017

Tournament-Синхрон северных стран
Spearman corr=0.8311346913783366
Kendall corr=0.6550434180139159

Tournament-Онлайн: Синхрон Урюбджирова
Spearman corr=0.7846813665210466
Kendall corr=0.6019902203220816

Tournament-Школьный Синхрон-lite. Выпуск 3.6
Spearman corr=0.860933622899259
Kendall corr=0.6868603058996536

Tournament-(а)Синхрон-lite. Лига старта. Эпизод XII
Spearman corr=0.7820539700167828
Kendall corr=0.6007176243364194

Tournament-Школьный Синхрон-lite. Выпуск 3.7
Spearman corr=0.8550240329059056
Kendall corr=0.6893718779075506

Tournament-Онлайн: (а)Синхрон-lite. Лига старта. Эпизод XIII
Spearman corr=0.7499456479463605
Kendall corr=0.5627736835884323

Tournament-Школьный Синхрон-lite. Выпуск 3.8
Spearman corr=0.6359578073820066
Kendall corr=0.4672769009551165

Tournament-Онлайн: (а)Синхрон-lite. Лига старта. Эпизод XIV
Spearman corr=0.6317414957381032
Kendall corr=0.46890999198975

Tournament-Онлайн: 19-30 Зелёный шум
Spearman corr=0.7448351067695265
Kendall corr=0.5828965086259651

Tournament-Онлайн: 19-30 Зелёный шум
Spearman corr=0.7089715869408764
Kendall corr=0.5621995873961274

Tournament-Онлайн: 16:00 Зелёный шум
Spearman corr=0.7387687194103584
Kendall corr=0.5855400437691199

Avg Spearman correlation=0.7793139742233105
Avg Kendall correlation=0.6257196634936709



Так как мы не знаем какой именно игрок ответил на тот или иной вопрос из команды, попробуем предсказать на сколько вероятно именно он ответил на заданный вопрос, введя тем самым латентные переменные для EM алгоритма. <br>
На Е шаге будем оценивать вероятность того, что именно рассматриваемый игрок ответил на заданный вопрос, учитывая силу каждого игрока из команды и сложность вопроса. При этом утверждаем, что игрок с нулевой вероятностью ответил на заданный вопрос, если вся команда не смогла на него ответить.<br>
На M шаге модель будет корректировать силу игроков и сложность вопросов, обучаясь на вероятностях, что именно рассматриваемый в наблюдении игрок ответил на заданный вопрос. 

In [149]:
tour_id = int
num_questions = int


class EM:
    def __init__(self, model, lr=10):
        self.model = model
        self.lr = lr
    
    def step_E(self, curr_player_id, question_id, team_member_ids):
        question_complexity = self.model.weights[question_id]
        cummulative_q = 1
        
        for member_id in team_member_ids:
            if member_id == curr_player_id:
                continue
            member_skill = self.model.weights[member_id]
            q = 1 - self.sigma(member_skill + question_complexity)
            cummulative_q *= q
        
        curr_member_skill = self.model.weights[curr_player_id]
        p = self.sigma(curr_member_skill + question_complexity)
        
        return p / (1 - cummulative_q + EPS)
    
    def step_M(self, X, y):
        self.model.fine_tune_params(X, y, self.lr)
    
    def init_updated_params(self, X, results, tournaments: Dict[tour_id, num_questions]):
        col_step = 0
        self.updated_params = []
        
        for tour_idx, size in tournaments.items():
            for team in results[tour_idx]:
                for q_idx, answer in enumerate(team['mask']):
                    team_member_ids = set([player_idx[member['player']['id']] 
                                           for member in team['teamMembers']])
                    for member in team['teamMembers']:
                        question_pos = len(player_idx) + col_step + q_idx
                        player_pos = player_idx[member['player']['id']]
                        self.updated_params.append((player_pos, question_pos, team_member_ids, answer))            
            col_step += size
    
    def train_epoch(self, X):
        y = []
        for player_id, question_id, team_member_ids, answer in tqdm(self.updated_params):
            #step_E
            if answer:
                hidden_state = self.step_E(player_id, question_id, team_member_ids)
            else:
                hidden_state = 0
                
            y.append(hidden_state)
            
        print("Start step M")
        y = np.array(y).clip(0, 1)
        self.step_M(X, y)
        
        return self.model
    
    @staticmethod
    def sigma(x):
        return 1 / (1 + np.exp(-x))

In [150]:
em_alg = EM(clf)

print("Start initialize train params")
em_alg.init_updated_params(X, results, train_tour_quest_quantity)
print("Finish")

Start initialize train params
Finish


    Сделаем 5 итерации, чтобы посмотреть растут ли метрики, модель обновляется градиентым спуском без батчей. В процессе обучения learning rate уменьшается

In [151]:
for epoch in range(5):
    print(f'{epoch=}')
    clf = em_alg.train_epoch(X)
    player_ratings = clf.weights[:len(player_idx)]
    calculate_average_correlations(results, player_ratings, list(test_tour_quest_quantity.keys()), False)

epoch=0


  0%|          | 0/21008480 [00:00<?, ?it/s]

Start step M


  0%|          | 0/50 [00:00<?, ?it/s]

LR=1.0
AVGLogLoss=1.2587905800949633
LR=0.1
AVGLogLoss=1.2583191412539552
LR=0.01
AVGLogLoss=1.2581258183390935
LR=0.001
AVGLogLoss=1.2581064909551773
LR=0.0001
AVGLogLoss=1.2581045582658592
Avg Spearman correlation=0.7793568683032638
Avg Kendall correlation=0.6258040297177829

epoch=1


...

LR=1.0
AVGLogLoss=1.2569616521141223
LR=0.1
AVGLogLoss=1.2564916801257433
LR=0.01
AVGLogLoss=1.2562989591528038
LR=0.001
AVGLogLoss=1.2562796919640569
LR=0.0001
AVGLogLoss=1.2562777652942636
Avg Spearman correlation=0.7792516587035001
Avg Kendall correlation=0.6256705001343864

epoch=4


  0%|          | 0/21008480 [00:00<?, ?it/s]

Start step M


  0%|          | 0/50 [00:00<?, ?it/s]

LR=1.0
AVGLogLoss=1.2563534088931556
LR=0.1
AVGLogLoss=1.2558839258114256
LR=0.01
AVGLogLoss=1.2556914054664463
LR=0.001
AVGLogLoss=1.255672158340775
LR=0.0001
AVGLogLoss=1.255670233677293
Avg Spearman correlation=0.7792292658700453
Avg Kendall correlation=0.6256299063720921



In [152]:
start_id = 0
tournament_complexity = []
for tournament_id, max_quest in train_tour_quest_quantity.items():
    tour_info = {}
    complexity = []
    for quest_id in range(max_quest):
        complexity.append(clf.weights[len(player_idx) + start_id + quest_id])
    tour_info['complexity'] = np.median(complexity)
    tour_info['id'] = tournament_id
    tour_info['name'] = tournaments.loc[tournament_id, 'name']
    tournament_complexity.append(tour_info)
    start_id += max_quest

tournament_ratings = pd.DataFrame(tournament_complexity)

Топ турниров по вопросам получился в обратную сторону, интересно...

In [153]:
tournament_ratings.sort_values(by=['complexity'], ascending=True).head(30)

,complexity,id,name
665,-4.451443,6149,Чемпионат Санкт-Петербурга. Первая лига
43,-2.171684,5159,Первенство правого полушария
543,-2.139705,5928,Угрюмый Ёрш
369,-1.861565,5684,Синхрон высшей лиги Москвы
641,-1.818074,6101,Воображаемый музей
225,-1.795673,5515,Чемпионат Минска. Лига А. Тур четвёртый
13,-1.729834,5025,Кубок городов
284,-1.711487,5587,Записки охотника
390,-1.681907,5717,Чемпионат Таджикистана
596,-1.665406,5996,Тихий Донец: омут первый


In [158]:
%%time
player_ratings = clf.weights[:len(player_idx)]
data = []

for player_id, num_corr_answers in train_player_ans_correct.items():
    player_info = players.loc[player_id]
    meta_info = {
        'rating': player_ratings[player_idx[player_id]],
        'true_answers': num_corr_answers,
        'name': player_info['name'],
        'surname': player_info['surname'],
        'patronymic': player_info['patronymic'],
    }
    data.append(meta_info)

pred_rating = pd.DataFrame(data)

Wall time: 3.64 s


In [159]:
pred_rating.sort_values(by=['rating'], ascending=False).head(30)

,rating,true_answers,name,surname,patronymic
8222,4.097260,1980,Максим,Руссо,Михайлович
5957,4.006010,2409,Александра,Брутер,Владимировна
1209,3.975121,3242,Иван,Семушин,Николаевич
155,3.776710,4000,Артём,Сорожкин,Сергеевич
1208,3.766705,2843,Михаил,Савченков,Владимирович
8244,3.766015,3202,Сергей,Спешков,Леонидович
1210,3.626611,1279,Станислав,Мереминский,Григорьевич
3,3.622141,1548,Сергей,Николенко,Игоревич
20010,3.548989,1172,Илья,Новиков,Сергеевич
5,3.548975,849,Ирина,Прокофьева,Сергеевна
